<a href="https://colab.research.google.com/github/tjrgh822/class_itsystem/blob/master/shuffle_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from __future__ import print_function
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K

from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator


(x_train, y_train), (x_test, y_test) = mnist.load_data()


datagen = ImageDataGenerator( width_shift_range=3, 
                             height_shift_range=3, 
                             horizontal_flip=True, 
                             vertical_flip=True,
                             brightness_range = [0.2, 1.0],
                             rotation_range = 90,
                             zoom_range = [0.5, 1.0]
                             )
                             

nodatagen = ImageDataGenerator()



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
x_train = np.load("/content/drive/My Drive/shuffledata_x.npy")
y_train = np.load("/content/drive/My Drive/shuffledata_y.npy")
y_train = np.argmax(y_train, axis=1).reshape(-1, 1)
y_train.shape

In [0]:

'''
def onehotencoding(data, case):
  encoding = np.eye(case)[data]
  return encoding.reshape(-1, case)'''

'''y_train = np.argmax(y_train, axis=1).reshape(-1, 1)
y_train.shape
argu_y_test = np.argmax(argu_y_test, axis=1).reshape(-1, 1)'''

argu= datagen.flow(x_test.reshape(-1, 28, 28, 1), y_test, batch_size = x_test.shape[0], shuffle=False)

argu_y_test = argu[0][1]
argu_x_test = argu[0][0].reshape(-1, 28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
argu_y_test = keras.utils.to_categorical(argu_y_test, num_classes)


In [0]:
print(f"{x_train.shape}, {y_train.shape}\n{argu_x_test.shape}, {argu_y_test.shape}")

In [0]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    argu_x_test = argu_x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    argu_x_test = argu_x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)





model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(28,28,1), kernel_size=(11,11), padding="valid", activation = "relu"))
# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(Dropout(0.25))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"))
# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(Dropout(0.25))


# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(units = 2048, activation = "relu"))
model.add(Dropout(0.25))


# 3rd Fully Connected Layer
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.25))

# Output Layer
model.add(Dense(10, activation = "softmax")) #As we have two classes

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()


In [0]:

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

batch_size = 100
epochs = 10

checkpointer = ModelCheckpoint(
    filepath='model_2.h5',
    save_best_only=True
)
early_stopping = EarlyStopping(patience=2)

callbacks=[checkpointer, early_stopping]

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch = x_train.shape[0]/ batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(argu_x_test, argu_y_test),
          validation_steps = y_train.shape[0]
          )

#score = model.evaluate(x_test, y_test, verbose=0)

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, LearningRateScheduler



In [0]:
!ls

In [0]:
from keras.models import load_model

model.save('model_2.h5')

from google.colab import files
files.download('model_2.h5')